In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/df2026/vnhsge/vnhsge_2022.csv
/kaggle/input/df2026/vnhsge/vnhsge_2021.csv
/kaggle/input/df2026/vnhsge/vnhsge_2024.csv
/kaggle/input/df2026/vnhsge/vnhsge_2020.csv
/kaggle/input/df2026/vnhsge/vnhsge_2023.csv
/kaggle/input/df2026/DATA/academic_records.csv
/kaggle/input/df2026/DATA/test.csv
/kaggle/input/df2026/DATA/admission.csv


In [2]:
df_2020 = pd.read_csv("/kaggle/input/df2026/vnhsge/vnhsge_2020.csv")
df_2021 = pd.read_csv("/kaggle/input/df2026/vnhsge/vnhsge_2021.csv")
df_2022 = pd.read_csv("/kaggle/input/df2026/vnhsge/vnhsge_2022.csv")
df_2023 = pd.read_csv("/kaggle/input/df2026/vnhsge/vnhsge_2023.csv")
df_2024 = pd.read_csv("/kaggle/input/df2026/vnhsge/vnhsge_2024.csv")

In [3]:
TARGET_COLS = [
    'SBD', 'MATH', 'LITERATURE', 'PHYSICS', 'CHEMISTRY', 'BIOLOGY', 
    'HISTORY', 'GEOGRAPHY', 'CIVIC_EDUCATION', 
    'FOREIGN_LANGUAGE', 'FOREIGN_LANGUAGE_CODE'
]

map_2020 = {
    'student_id': 'SBD',
    'mathematics': 'MATH',
    'literature': 'LITERATURE',
    'physics': 'PHYSICS',
    'chemistry': 'CHEMISTRY',
    'biology': 'BIOLOGY',
    'history': 'HISTORY',
    'geography': 'GEOGRAPHY',
    'civic_education': 'CIVIC_EDUCATION',
    'foreign_language_score': 'FOREIGN_LANGUAGE',
    'foreign_language_code': 'FOREIGN_LANGUAGE_CODE'
}

map_2021 = {
    'id_examinee': 'SBD',
    'math': 'MATH',
    'literature': 'LITERATURE',
    'physics': 'PHYSICS',
    'chemistry': 'CHEMISTRY',
    'biology': 'BIOLOGY',
    'history': 'HISTORY',
    'geography': 'GEOGRAPHY',
    'civic_education': 'CIVIC_EDUCATION'
}

map_2022_2023_2024 = {
    'sbd': 'SBD',
    'toan': 'MATH',
    'ngu_van': 'LITERATURE',
    'vat_li': 'PHYSICS',
    'hoa_hoc': 'CHEMISTRY',
    'sinh_hoc': 'BIOLOGY',
    'lich_su': 'HISTORY',
    'dia_li': 'GEOGRAPHY',
    'gdcd': 'CIVIC_EDUCATION',
    'ngoai_ngu': 'FOREIGN_LANGUAGE',
    'ma_ngoai_ngu': 'FOREIGN_LANGUAGE_CODE'
}

def process_data(df, mapping_dict):
    df = df.rename(columns=mapping_dict)

    if 'english' in df.columns:
        lang_cols = ['english', 'russian', 'french', 'chinese', 'german', 'japanese']
        lang_codes = ['N1', 'N2', 'N3', 'N4', 'N5', 'N6']
        
        valid_lang_cols = [c for c in lang_cols if c in df.columns]
        if valid_lang_cols:
            df['FOREIGN_LANGUAGE'] = df[valid_lang_cols].max(axis=1)
            conditions = [df[col].notna() for col in lang_cols]
            df['FOREIGN_LANGUAGE_CODE'] = np.select(conditions, lang_codes, default=None)

    if 'FOREIGN_LANGUAGE_CODE' not in df.columns:
        df['FOREIGN_LANGUAGE_CODE'] = np.nan

    df['FOREIGN_LANGUAGE_CODE'] = df['FOREIGN_LANGUAGE_CODE'].astype('object')
        
    missing_code_mask = (df['FOREIGN_LANGUAGE'].notna()) & (df['FOREIGN_LANGUAGE_CODE'].isna())
    if missing_code_mask.any():
        df.loc[missing_code_mask, 'FOREIGN_LANGUAGE_CODE'] = 'N1'
        
    for col in TARGET_COLS:
        if col not in df.columns:
            df[col] = np.nan 
            
    return df[TARGET_COLS]

df_2020_clean = process_data(df_2020, map_2020)
df_2021_clean = process_data(df_2021, map_2021)
df_2022_clean = process_data(df_2022, map_2022_2023_2024)
df_2023_clean = process_data(df_2023, map_2022_2023_2024)
df_2024_clean = process_data(df_2024, map_2022_2023_2024)

In [4]:
def get_information(df, combo_list):
    FORMULAS = {
        'A00': {'cols': ['MATH', 'PHYSICS', 'CHEMISTRY'], 'lang_req': None},
        'B00': {'cols': ['MATH', 'CHEMISTRY', 'BIOLOGY'], 'lang_req': None},
        'A01': {'cols': ['MATH', 'PHYSICS', 'FOREIGN_LANGUAGE'], 'lang_req': 'N1'},
        'D01': {'cols': ['MATH', 'LITERATURE', 'FOREIGN_LANGUAGE'], 'lang_req': 'N1'},
        'D07': {'cols': ['MATH', 'CHEMISTRY', 'FOREIGN_LANGUAGE'], 'lang_req': 'N1'},
        'D29': {'cols': ['MATH', 'PHYSICS', 'FOREIGN_LANGUAGE'], 'lang_req': 'N3'}, 
        'D24': {'cols': ['MATH', 'CHEMISTRY', 'FOREIGN_LANGUAGE'], 'lang_req': 'N3'}
    }
    
    results = {}
    valid_scores_pool = [] 
    missing_combos_flag = False 

    for combo in combo_list:
        if combo in FORMULAS:
            config = FORMULAS[combo]
            cols = config['cols']
            lang_req = config['lang_req']
            
            if set(cols).issubset(df.columns):
                if lang_req:
                    df_filtered = df[df['FOREIGN_LANGUAGE_CODE'] == lang_req]
                else:
                    df_filtered = df
                
                if not df_filtered.empty:
                    total_series = df_filtered[cols].sum(axis=1, min_count=len(cols))
                    total_series = total_series.dropna()
                    
                    if not total_series.empty:
                        stats = {
                            'mean': total_series.mean(),
                            'std': total_series.std(),
                            'median': total_series.median()
                        }
                        results[combo] = stats
                        valid_scores_pool.append(total_series)
                    else:
                        missing_combos_flag = True
                else:
                    missing_combos_flag = True
            else:
                missing_combos_flag = True
        else:
            missing_combos_flag = True

    if missing_combos_flag:
        if valid_scores_pool:
            all_valid_scores = pd.concat(valid_scores_pool)
            results['OTHER'] = {
                'mean': all_valid_scores.mean(),
                'std': all_valid_scores.std(),
                'median': all_valid_scores.median()
            }
        else:
            results['OTHER'] = {'mean': 0, 'std': 0, 'median': 0}

    return results

In [5]:
to_hop = ['A00', 'A01', 'B00', 'D01', 'D07', 'D24', 'D29', 'H01', 'K00', 'K01', 'K02', 'TT', 'V00', 'V01', 'V02', 'V06', 'V10', 'X01', 'X03', 'X05', 'X06']
stats = get_information(df_2022_clean, to_hop)
print(stats)

{'A00': {'mean': np.float64(21.095522023770904), 'std': 3.2378389978733377, 'median': 21.45}, 'A01': {'mean': np.float64(20.290872906549115), 'std': 3.339589421960257, 'median': 20.4}, 'B00': {'mean': np.float64(19.403933112714352), 'std': 3.1554830222785997, 'median': 19.55}, 'D01': {'mean': np.float64(18.43807900961083), 'std': 3.884606861658652, 'median': 18.55}, 'D07': {'mean': np.float64(20.23967531840988), 'std': 3.20641673945586, 'median': 20.5}, 'OTHER': {'mean': np.float64(19.519640781958874), 'std': 3.6548108151510372, 'median': 19.799999999999997}}
